In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime, timedelta
import netCDF4 as nc
import glob
import numpy as np
import cartopy.crs as ccrs   
from matplotlib.pyplot import figure
from functools import partial

from tqdm.notebook import tqdm
from idf_analysis import IntensityDurationFrequencyAnalyse
from idf_analysis.definitions import *
from os import path

In [ ]:
def read_netcdfs(files, transform_func=None):
    def process_one_path(path):
        with xr.open_dataset(path, drop_variables=['time_bounds', 'lat_bounds', 'lon_bounds']) as ds:
            if transform_func is not None:
                ds = transform_func(ds)

            ds.load()
            return ds

    paths = sorted(glob.glob(files))
    datasets = [process_one_path(p) for p in paths]
    combined = xr.combine_by_coords(datasets)
    return combined
# 18.655297, -33.991888
# -0.762 ~ +0.574, -0.762 ~ +0.574
# malabon
lon_bounds, lat_bounds = (17.893297, 19.229297), (-34.753888, -33.417888)
combined = read_netcdfs('/gucnas2/emir/cmorph/*/*/*/*.nc',
                        transform_func=lambda ds: ds.sel(lon=slice(*lon_bounds), lat=slice(*lat_bounds)).mean(dim=['lat', 'lon']))
combined

# Save to yaml file

In [ ]:
# # Run this so you dont need to process the whole dataset again. Read the yaml file instead which is much smaller
idf = IntensityDurationFrequencyAnalyse(series_kind=SERIES.PARTIAL, worksheet=METHOD.KOSTRA, extended_durations=True)
time = pd.DatetimeIndex(combined['time'])
cmorph = pd.Series(combined['cmorph'], index=time)

idf.set_series(pd.Series(cmorph))
idf.auto_save_parameters(path.join('/gucnas2/emir/cmorph', 'kisumu_idf_parameters.yaml'))

# Read saved yaml file

In [ ]:
idf = IntensityDurationFrequencyAnalyse(series_kind=SERIES.PARTIAL, worksheet=METHOD.KOSTRA, extended_durations=True)
idf.read_parameters('/gucnas2/emir/cmorph/accra_idf_parameters.yaml')

In [ ]:
idf.result_table(add_names=True).round(2)
idf.rain_events

In [ ]:
idf.result_table(add_names=True).round(2).to_csv(path.join('/gucnas2/emir/cmorph', 'kisumu_idf.csv'), sep=',', decimal='.', float_format='%0.2f')

In [ ]:
fig, ax = idf.result_figure(color=True, add_interim=False)

In [ ]:
# Calculate average rainfall per year

time = pd.DatetimeIndex(combined['time'])
years = combined.groupby('time.year').sum()

annual_average = years.cmorph[:-1].mean()
annual_average/2

# WRF Heatparams

wrf_heat = xr.open_dataset('/gucnas2/emir/wrf/kisumu/wrfout_d03_2023-02-01_00:00:00_heatparams.nc')
wrf_heat.UTCI.mean(dim=['Time', 'south_north', 'west_east'])